<a href="https://colab.research.google.com/github/roveen001/gfa/blob/main/gfa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global Food Analysis on Staple Food Project
## Dashboard

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
# read all the necessary files
# meat production
meat_pn = pd.read_csv('drive/MyDrive/Colab Notebooks/gfa/global-meat-production.csv')
# wheat production
wheat_pn = pd.read_csv('drive/MyDrive/Colab Notebooks/gfa/wheat-production.csv')
# rice production
rice_pn = pd.read_csv('drive/MyDrive/Colab Notebooks/gfa/rice-production.csv')
# orange production
orange_pn = pd.read_csv('drive/MyDrive/Colab Notebooks/gfa/orange-production.csv')

In [3]:
# GDP 
gdp = pd.read_csv('drive/MyDrive/Colab Notebooks/gfa/annual-food-expenditure-per-person-vs-gdp-per-capita.csv' )

In [5]:
# Food pricing 
fp = pd.read_csv('drive/MyDrive/Colab Notebooks/gfa/global_food_prices.csv', low_memory=False)
fp

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0000,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0000,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0000,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0000,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050633,271.0,Zimbabwe,3444,Midlands,5594,Mbilashaba,432,Beans (sugar) - Retail,0.0,ZWL,15,Retail,5,KG,6,2021,233.3333,NaN
2050634,271.0,Zimbabwe,3444,Midlands,5594,Mbilashaba,539,Toothpaste - Retail,0.0,ZWL,15,Retail,116,100 ML,6,2021,112.5000,NaN
2050635,271.0,Zimbabwe,3444,Midlands,5594,Mbilashaba,540,Laundry soap - Retail,0.0,ZWL,15,Retail,5,KG,6,2021,114.0000,NaN
2050636,271.0,Zimbabwe,3444,Midlands,5594,Mbilashaba,541,Handwash soap - Retail,0.0,ZWL,15,Retail,66,250 G,6,2021,59.5000,NaN


## Wheat

In [9]:
# lets get the price wheat r'\bwheat gmi'
wheat_pr = fp.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
wheat_pr = wheat_pr[wheat_pr['cm_name'].str.match('^Wheat.*') == True]
wheat_pr = wheat_pr.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
wheat_pr

,adm0_name,cm_name,mp_year,mp_price
0,Afghanistan,Wheat - Retail,2000,12.116458
1,Afghanistan,Wheat - Retail,2001,10.286000
2,Afghanistan,Wheat - Retail,2002,5.923125
3,Afghanistan,Wheat - Retail,2003,7.089340
4,Afghanistan,Wheat - Retail,2004,9.061580
...,...,...,...,...
1231,Zimbabwe,Wheat - Retail,2015,0.485000
1232,Zimbabwe,Wheat - Retail,2016,0.500769
1233,Zimbabwe,Wheat - Retail,2017,0.550000
1234,Zimbabwe,Wheat flour (fortified) - Retail,2017,1.126316


In [10]:
# finsh the table for wheat
wheat_pr = wheat_pr.merge(wheat_pn,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])
wheat_pr = wheat_pr.merge(gdp,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])

In [11]:
wheat_pr

,adm0_name,cm_name,mp_year,mp_price,Entity_x,Code_x,Year_x,Wheat | 00000015 || Production | 005510 || tonnes,Entity_y,Code_y,Year_y,GDP per capita (current US$),Food expenditure (USDA (2017)),Population (historical estimates),Continent
0,Afghanistan,Wheat - Retail,2000,12.116458,Afghanistan,AFG,2000,1469000,Afghanistan,AFG,2000,NaN,NaN,19542986.0,NaN
1,Afghanistan,Wheat flour - Retail,2000,14.202917,Afghanistan,AFG,2000,1469000,Afghanistan,AFG,2000,NaN,NaN,19542986.0,NaN
2,Afghanistan,Wheat - Retail,2001,10.286000,Afghanistan,AFG,2001,1597000,Afghanistan,AFG,2001,NaN,NaN,19688634.0,NaN
3,Afghanistan,Wheat flour - Retail,2001,11.760889,Afghanistan,AFG,2001,1597000,Afghanistan,AFG,2001,NaN,NaN,19688634.0,NaN
4,Afghanistan,Wheat - Retail,2002,5.923125,Afghanistan,AFG,2002,2686000,Afghanistan,AFG,2002,179.426575,NaN,21000258.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,Zimbabwe,Wheat - Retail,2015,0.485000,Zimbabwe,ZWE,2015,45302,Zimbabwe,ZWE,2015,1445.069702,NaN,14154937.0,Africa
804,Zimbabwe,Wheat - Retail,2016,0.500769,Zimbabwe,ZWE,2016,42008,Zimbabwe,ZWE,2016,1464.588989,NaN,14452705.0,NaN
805,Zimbabwe,Wheat - Retail,2017,0.550000,Zimbabwe,ZWE,2017,38715,Zimbabwe,ZWE,2017,1235.189087,NaN,14751101.0,NaN
806,Zimbabwe,Wheat flour (fortified) - Retail,2017,1.126316,Zimbabwe,ZWE,2017,38715,Zimbabwe,ZWE,2017,1235.189087,NaN,14751101.0,NaN


In [12]:
# insert new column
wheat_pr['wheat_af'] = ((wheat_pr['Wheat | 00000015 || Production | 005510 || tonnes']/wheat_pr['Population (historical estimates)'] * wheat_pr['mp_price'])/ wheat_pr['GDP per capita (current US$)']) *100
wheat_pr = wheat_pr.drop(columns=['Entity_x','Code_x','Year_x','Entity_y','Code_y','Year_y','Food expenditure (USDA (2017))','Continent'])

## Rice

In [13]:
# lets get the price rice 'r`\bwheat`gmi'
rice_pr = fp.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
rice_pr = rice_pr[rice_pr['cm_name'].str.match('^Rice.*') == True]
rice_pr = rice_pr.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
rice_pr

,adm0_name,cm_name,mp_year,mp_price
0,Afghanistan,Rice (high quality) - Retail,2020,87.353554
1,Afghanistan,Rice (high quality) - Retail,2021,88.027591
2,Afghanistan,Rice (low quality) - Retail,2007,27.249533
3,Afghanistan,Rice (low quality) - Retail,2008,40.336394
4,Afghanistan,Rice (low quality) - Retail,2009,34.720331
...,...,...,...,...
2672,Zambia,Rice (local) - Retail,2019,15.919710
2673,Zambia,Rice (local) - Retail,2020,19.172181
2674,Zambia,Rice (local) - Retail,2021,24.355268
2675,Zimbabwe,Rice - Retail,2020,103.700241


In [14]:
rice_pr=rice_pr.merge(rice_pn,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])
rice_pr = rice_pr.merge(gdp,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])
rice_pr

,adm0_name,cm_name,mp_year,mp_price,Entity_x,Code_x,Year_x,Rice | 00000027 || Production | 005510 || tonnes,Entity_y,Code_y,Year_y,GDP per capita (current US$),Food expenditure (USDA (2017)),Population (historical estimates),Continent
0,Afghanistan,Rice (high quality) - Retail,2020,87.353554,Afghanistan,AFG,2020,439549,Afghanistan,AFG,2020,516.747864,NaN,38972236.0,NaN
1,Afghanistan,Rice (low quality) - Retail,2020,48.948244,Afghanistan,AFG,2020,439549,Afghanistan,AFG,2020,516.747864,NaN,38972236.0,NaN
2,Afghanistan,Rice (low quality) - Retail,2007,27.249533,Afghanistan,AFG,2007,552000,Afghanistan,AFG,2007,359.693146,NaN,25903306.0,NaN
3,Afghanistan,Rice (low quality) - Retail,2008,40.336394,Afghanistan,AFG,2008,612000,Afghanistan,AFG,2008,364.663544,NaN,26427204.0,NaN
4,Afghanistan,Rice (low quality) - Retail,2009,34.720331,Afghanistan,AFG,2009,645000,Afghanistan,AFG,2009,437.268738,NaN,27385310.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,Zambia,Rice (local) - Retail,2017,14.377926,Zambia,ZMB,2017,38423,Zambia,ZMB,2017,1535.196533,NaN,17298056.0,NaN
2108,Zambia,Rice (local) - Retail,2018,14.057028,Zambia,ZMB,2018,43063,Zambia,ZMB,2018,1516.368408,NaN,17835898.0,NaN
2109,Zambia,Rice (local) - Retail,2019,15.919710,Zambia,ZMB,2019,29584,Zambia,ZMB,2019,1305.000977,NaN,18380478.0,NaN
2110,Zambia,Rice (local) - Retail,2020,19.172181,Zambia,ZMB,2020,34630,Zambia,ZMB,2020,985.132446,NaN,18927716.0,NaN


In [15]:
rice_pr['rice_af'] = ((rice_pr['Rice | 00000027 || Production | 005510 || tonnes']/rice_pr['Population (historical estimates)'] * rice_pr['mp_price'])/ rice_pr['GDP per capita (current US$)']) *100
rice_pr = rice_pr.drop(columns=['Entity_x','Code_x','Year_x','Entity_y','Code_y','Year_y','Food expenditure (USDA (2017))','Continent'])

## Meat 

In [16]:
# lets get the price meat 'r`\bwheat`gmi'
meat_pr = fp.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
meat_pr = meat_pr[meat_pr['cm_name'].str.match('^Meat.*') == True]
meat_pr = meat_pr.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
meat_pr

,adm0_name,cm_name,mp_year,mp_price
0,Algeria,Meat (beef) - Retail,2015,1200.000000
1,Algeria,Meat (camel) - Retail,2015,614.814815
2,Algeria,Meat (camel) - Retail,2016,669.459459
3,Algeria,Meat (chicken) - Retail,2015,348.166667
4,Algeria,Meat (chicken) - Retail,2016,338.954545
...,...,...,...,...
1129,Yemen,Meat (mutton) - Retail,2016,2726.820000
1130,Yemen,Meat (mutton) - Retail,2017,2937.050000
1131,Yemen,Meat (mutton) - Retail,2018,3834.083333
1132,Yemen,Meat (mutton) - Retail,2019,5176.936170


In [17]:
meat_pr = meat_pr.merge(meat_pn,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])
meat_pr = meat_pr.merge(gdp,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])
meat_pr

,adm0_name,cm_name,mp_year,mp_price,Entity_x,Code_x,Year_x,"Meat, total | 00001765 || Production | 005510 || tonnes",Entity_y,Code_y,Year_y,GDP per capita (current US$),Food expenditure (USDA (2017)),Population (historical estimates),Continent
0,Algeria,Meat (beef) - Retail,2015,1200.000000,Algeria,DZA,2015,771661,Algeria,DZA,2015,4177.889648,724.0,39543148.0,Africa
1,Algeria,Meat (camel) - Retail,2015,614.814815,Algeria,DZA,2015,771661,Algeria,DZA,2015,4177.889648,724.0,39543148.0,Africa
2,Algeria,Meat (chicken) - Retail,2015,348.166667,Algeria,DZA,2015,771661,Algeria,DZA,2015,4177.889648,724.0,39543148.0,Africa
3,Algeria,Meat (camel) - Retail,2016,669.459459,Algeria,DZA,2016,801747,Algeria,DZA,2016,3946.452393,639.0,40339328.0,NaN
4,Algeria,Meat (chicken) - Retail,2016,338.954545,Algeria,DZA,2016,801747,Algeria,DZA,2016,3946.452393,639.0,40339328.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,Yemen,Meat (mutton) - Retail,2018,3834.083333,Yemen,YEM,2018,361224,Yemen,YEM,2018,758.145264,NaN,30790514.0,NaN
865,Yemen,Meat (chicken) - Retail,2019,1577.506383,Yemen,YEM,2019,400104,Yemen,YEM,2019,NaN,NaN,31546694.0,NaN
866,Yemen,Meat (mutton) - Retail,2019,5176.936170,Yemen,YEM,2019,400104,Yemen,YEM,2019,NaN,NaN,31546694.0,NaN
867,Yemen,Meat (chicken) - Retail,2020,1671.875000,Yemen,YEM,2020,417391,Yemen,YEM,2020,NaN,NaN,32284044.0,NaN


In [18]:
meat_pr['meat_af'] = ((meat_pr['Meat, total | 00001765 || Production | 005510 || tonnes']/meat_pr['Population (historical estimates)'] * meat_pr['mp_price'])/ meat_pr['GDP per capita (current US$)']) *100
meat_pr = meat_pr.drop(columns=['Entity_x','Code_x','Year_x','Entity_y','Code_y','Year_y','Food expenditure (USDA (2017))','Continent'])

## Orange

In [19]:
# lets get the orange price 'r`\bwheat`gmi'
orange_pr = fp.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
orange_pr = orange_pr[orange_pr['cm_name'].str.match('^Orange.*') == True]
orange_pr = orange_pr.groupby(['adm0_name','cm_name','mp_year'],as_index=False)['mp_price'].mean()
orange_pr

,adm0_name,cm_name,mp_year,mp_price
0,Algeria,Oranges - Retail,2015,220.875000
1,Benin,Oranges - Retail,2019,197.134023
2,Benin,Oranges - Retail,2020,205.483706
3,Benin,Oranges - Retail,2021,213.475604
4,Colombia,Oranges (big size) - Wholesale,2013,689.000000
...,...,...,...,...
66,Turkey,Oranges - Retail,2017,2.852100
67,Turkey,Oranges - Retail,2018,2.982422
68,Turkey,Oranges - Retail,2019,4.339036
69,Turkey,Oranges - Retail,2020,5.369764


In [20]:
orange_pr = orange_pr.merge(orange_pn,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])
orange_pr = orange_pr.merge(gdp,left_on=['adm0_name','mp_year'],right_on=['Entity','Year'])
orange_pr

,adm0_name,cm_name,mp_year,mp_price,Entity_x,Code_x,Year_x,Oranges | 00000490 || Production | 005510 || tonnes,Entity_y,Code_y,Year_y,GDP per capita (current US$),Food expenditure (USDA (2017)),Population (historical estimates),Continent
0,Algeria,Oranges - Retail,2015,220.875000,Algeria,DZA,2015,1005079,Algeria,DZA,2015,4177.889648,724.0,39543148.0,Africa
1,Benin,Oranges - Retail,2019,197.134023,Benin,BEN,2019,14859,Benin,BEN,2019,1219.515503,NaN,12290443.0,NaN
2,Benin,Oranges - Retail,2020,205.483706,Benin,BEN,2020,14906,Benin,BEN,2020,1291.041016,NaN,12643121.0,NaN
3,Colombia,Oranges (big size) - Wholesale,2013,689.000000,Colombia,COL,2013,245267,Colombia,COL,2013,8218.347656,NaN,46237928.0,NaN
4,Colombia,Oranges (big size) - Wholesale,2014,835.166667,Colombia,COL,2014,233342,Colombia,COL,2014,8114.343750,NaN,46677944.0,NaN
5,Colombia,Oranges (big size) - Wholesale,2015,1030.333333,Colombia,COL,2015,268750,Colombia,COL,2015,6175.875977,650.0,47119728.0,South America
6,Colombia,Oranges (big size) - Wholesale,2016,1174.791667,Colombia,COL,2016,294895,Colombia,COL,2016,5870.777832,646.0,47625960.0,NaN
7,Colombia,Oranges (big size) - Wholesale,2017,1005.833333,Colombia,COL,2017,331502,Colombia,COL,2017,6376.706543,NaN,48351668.0,NaN
8,Colombia,Oranges (big size) - Wholesale,2018,1031.818182,Colombia,COL,2018,338418,Colombia,COL,2018,6729.583496,NaN,49276968.0,NaN
9,Colombia,Oranges (big size) - Wholesale,2019,1163.333333,Colombia,COL,2019,351200,Colombia,COL,2019,6424.979492,NaN,50187404.0,NaN


In [21]:
orange_pr['orange_af'] = ((orange_pr['Oranges | 00000490 || Production | 005510 || tonnes']/orange_pr['Population (historical estimates)'] * orange_pr['mp_price'])/ orange_pr['GDP per capita (current US$)']) *100
orange_pr = orange_pr.drop(columns=['Entity_x','Code_x','Year_x','Entity_y','Code_y','Year_y','Food expenditure (USDA (2017))','Continent'])

## Affordibility

In [22]:
# lets define the class that holds the name of the countries we are interesed in
countries = {0:'Nepal',
             1:'Colombia',
             2:'Turkey',
             3:'Ethopita',
             4:'Jordan',
             5:'Mali',
             6:'Nigeria',
             7:'Rowanda',
             8:'State of Palestine'}
ser = pd.Series(data=countries)

In [23]:
# create a df to hold the data that we need using concat
ft = pd.concat([wheat_pr,rice_pr, meat_pr, orange_pr],ignore_index=True)
ft = ft.loc[ft.adm0_name.isin(ser)]
ft

,adm0_name,cm_name,mp_year,mp_price,Wheat | 00000015 || Production | 005510 || tonnes,GDP per capita (current US$),Population (historical estimates),wheat_af,Rice | 00000027 || Production | 005510 || tonnes,rice_af,"Meat, total | 00001765 || Production | 005510 || tonnes",meat_af,Oranges | 00000490 || Production | 005510 || tonnes,orange_af
248,Colombia,Wheat - Wholesale,2020,7563.327966,6295.0,5334.556152,50930656.0,0.017524,NaN,NaN,NaN,NaN,NaN,NaN
249,Colombia,Wheat flour (local) - Wholesale,2020,7801.062346,6295.0,5334.556152,50930656.0,0.018075,NaN,NaN,NaN,NaN,NaN,NaN
250,Colombia,Wheat flour - Wholesale,2020,2245.967765,6295.0,5334.556152,50930656.0,0.005204,NaN,NaN,NaN,NaN,NaN,NaN
251,Colombia,Wheat flour - Wholesale,2000,739.197083,37085.0,2520.481201,39215136.0,0.027735,NaN,NaN,NaN,NaN,NaN,NaN
252,Colombia,Wheat flour - Wholesale,2001,792.915833,31793.0,2439.682373,39837876.0,0.025938,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3828,Turkey,Oranges - Retail,2016,1.621250,NaN,10894.603516,81019400.0,NaN,NaN,NaN,NaN,NaN,1850000.0,0.000340
3829,Turkey,Oranges - Retail,2017,2.852100,NaN,10589.667969,82089824.0,NaN,NaN,NaN,NaN,NaN,1950000.0,0.000640
3830,Turkey,Oranges - Retail,2018,2.982422,NaN,9454.348633,82809304.0,NaN,NaN,NaN,NaN,NaN,1900000.0,0.000724
3831,Turkey,Oranges - Retail,2019,4.339036,NaN,9121.515625,83481688.0,NaN,NaN,NaN,NaN,NaN,1700000.0,0.000969


In [24]:
# export the csv file 
ft.to_csv('drive/MyDrive/Colab Notebooks/gfa/final_tb.csv')

### see for 2021

In [29]:
gk = ft.groupby(['adm0_name','mp_year'])
gk.first()

cm_name     mp_price  \
adm0_name mp_year                                         
Colombia  2000     Wheat flour - Wholesale   739.197083   
          2001     Wheat flour - Wholesale   792.915833   
          2002     Wheat flour - Wholesale   855.857921   
          2003     Wheat flour - Wholesale  1058.634587   
          2004     Wheat flour - Wholesale  1113.225417   
...                                    ...          ...   
Turkey    2016        Wheat flour - Retail     2.848333   
          2017        Wheat flour - Retail     3.132718   
          2018        Wheat flour - Retail     3.405498   
          2019        Wheat flour - Retail     3.685917   
          2020        Wheat flour - Retail     4.546042   

                   Wheat | 00000015 || Production | 005510 || tonnes  \
adm0_name mp_year                                                      
Colombia  2000                                               37085.0   
          2001                                               31793.0   
          2002                                               40795.0   
          2003                                               46571.0   
          2004                                               43471.0   
...                                                              ...   
Turkey    2016                                            20600000.0   
          2017                                            21500000.0   
          2018                                            20000000.0   
          2019                                            19000000.0   
          2020                                            20500000.0   

                   GDP per capita (current US$)  \
adm0_name mp_year                                 
Colombia  2000                      2520.481201   
          2001                      2439.682373   
          2002                      2396.627197   
          2003                      2281.401855   
          2004                      2782.623291   
...                                         ...   
Turkey    2016                     10894.603516   
          2017                     10589.667969   
          2018                      9454.348633   
          2019                      9121.515625   
          2020                      8536.433594   

                   Population (historical estimates)  wheat_af  \
adm0_name mp_year                                                
Colombia  2000                            39215136.0  0.027735   
          2001                            39837876.0  0.025938   
          2002                            40454056.0  0.036012   
          2003                            41057688.0  0.052634   
          2004                            41648272.0  0.041757   
...                                              ...       ...   
Turkey    2016                            81019400.0  0.006647   
          2017                            82089824.0  0.007748   
          2018                            82809304.0  0.008700   
          2019                            83481688.0  0.009197   
          2020                            84135432.0  0.012976   

                   Rice | 00000027 || Production | 005510 || tonnes   rice_af  \
adm0_name mp_year                                                               
Colombia  2000                                            2237270.0  2.173670   
          2001                                            2156310.0  2.392302   
          2002                                            2063030.0  2.339390   
          2003                                            2417190.0  3.186373   
          2004                                            2496720.0  2.721922   
...                                                             ...       ...   
Turkey    2016                                             920000.0  0.000630   
          2017                                             900000.0  0.000742 